In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
# simplified : replace the stem structure with one convolutional layer, channels are divided by 4

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import os
import numpy as np
import math

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 150
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0001
checkpoint_save_path =  './Model/InceptionResnetV2/'
log_dir = os.path.join("Model","InceptionResnetV2_logs")

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.125,
               height_shift_range = 0.125,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.125,
               zoom_range = 0.125)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):  #HTD(-6,3)衰减
    start = -6.0
    end = 3.0
    return lr / 2.0 * (1- math.tanh( (end-start)*epoch/epochs + start))

In [ ]:
class ConvBNRelu(Model):
    def __init__(self,channels,kernel_size,strides,padding):
        super(ConvBNRelu,self).__init__()
        self.model = tf.keras.models.Sequential([
                    Conv2D(filters=channels, kernel_size=kernel_size,strides=strides, padding=padding,
                           kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER)),
                    BatchNormalization(momentum=0.9),
                    Activation('relu') ])
    def call(self,inputs):
        outputs = self.model(inputs)
        return outputs

In [ ]:
class InceptionResnetA(Model):
    def __init__(self,channels=[32,32,32,32,48,64,384]):
        super(InceptionResnetA,self).__init__()
        channels=[int(i/4) for i in channels]
        self.c1 = ConvBNRelu(channels=channels[0],kernel_size=1,strides=1, padding='same')
        self.c2_1 = ConvBNRelu(channels=channels[1],kernel_size=1,strides=1, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels[2],kernel_size=3,strides=1, padding='same')
        self.c3_1 = ConvBNRelu(channels=channels[3],kernel_size=1,strides=1, padding='same')
        self.c3_2 = ConvBNRelu(channels=channels[4],kernel_size=3,strides=1, padding='same')
        self.c3_3 = ConvBNRelu(channels=channels[5],kernel_size=3,strides=1, padding='same')
        self.c4 =  Conv2D(filters=channels[6], kernel_size=1,strides=1, padding='same',
                           kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
    def call(self,inputs):
        x1 = self.c1(inputs)
        x2_1 = self.c2_1(inputs)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(inputs)
        x3_2 = self.c3_2(x3_1)
        x3_3 = self.c3_3(x3_2)
        x = tf.concat([x1,x2_2,x3_3], axis = -1)
        y = self.c4(x)
        outputs = self.a1(self.b1(y + inputs))
        return outputs

In [ ]:
class InceptionResnetB(Model):
    def __init__(self,channels = [192,128,160,192,1152]):
        super(InceptionResnetB,self).__init__()
        channels=[int(i/4) for i in channels]
        self.c1 = ConvBNRelu(channels=channels[0],kernel_size=1,strides=1, padding='same')
        self.c2_1 = ConvBNRelu(channels=channels[1],kernel_size=1,strides=1, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels[2],kernel_size=(1,7),strides=1, padding='same')
        self.c2_3 = ConvBNRelu(channels=channels[3],kernel_size=(7,1),strides=1, padding='same')
        self.c3 =  Conv2D(filters=channels[4], kernel_size=1,strides=1, padding='same',
                          kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
    def call(self,inputs):      
        x1 = self.c1(inputs)
        x2_1 = self.c2_1(inputs)
        x2_2 = self.c2_2(x2_1)
        x2_3 = self.c2_3(x2_2)
        x = tf.concat([x1,x2_3], axis = -1)
        y = self.c3(x)
        outputs = self.a1(self.b1(y + inputs))
        return outputs

In [ ]:
class InceptionResnetC(Model):
    def __init__(self,channels = [192,192,224,256,2144]):
        super(InceptionResnetC,self).__init__() 
        channels=[int(i/4) for i in channels]
        self.c1 = ConvBNRelu(channels=channels[0],kernel_size=1,strides=1, padding='same')
        self.c2_1 = ConvBNRelu(channels=channels[1],kernel_size=1,strides=1, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels[2],kernel_size=(1,3),strides=1, padding='same')
        self.c2_3 = ConvBNRelu(channels=channels[3],kernel_size=(3,1),strides=1, padding='same')
        self.c3 =  Conv2D(filters=channels[4], kernel_size=1,strides=1, padding='same',
                          kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))
        self.b1 = BatchNormalization(momentum=0.9)
        self.a1 = Activation('relu')
    def call(self,inputs):
        x1 = self.c1(inputs)
        x2_1 = self.c2_1(inputs)
        x2_2 = self.c2_2(x2_1)
        x2_3 = self.c2_3(x2_2)
        x = tf.concat([x1,x2_3], axis = -1)
        y = self.c3(x)
        outputs = self.a1(self.b1(y + inputs))
        return outputs

In [ ]:
class InceptionReduceA(Model):
    def __init__(self,channels = [384,256,256,384]):
        super(InceptionReduceA,self).__init__()    
        channels=[int(i/4) for i in channels]
        self.c1_1 = ConvBNRelu(channels=channels[0],kernel_size=3,strides=2, padding='valid')
        self.c2_1 = ConvBNRelu(channels=channels[1],kernel_size=1,strides=1, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels[2],kernel_size=3,strides=1, padding='same')
        self.c2_3 = ConvBNRelu(channels=channels[3],kernel_size=3,strides=2, padding='valid')
        self.p3_1 = MaxPool2D(pool_size=3,strides=2, padding='valid')
    def call(self,inputs):
        x1_1 = self.c1_1(inputs)
        x2_1 = self.c2_1(inputs)
        x2_2 = self.c2_2(x2_1)
        x2_3 = self.c2_3(x2_2)
        x3_1 = self.p3_1(inputs)
        outputs = tf.concat([x1_1,x2_3,x3_1], axis = -1)
        return outputs

In [ ]:
class InceptionReduceB(Model):
    def __init__(self,channels = [256,384,256,288,256,288,320]):
        super(InceptionReduceB,self).__init__()       
        channels=[int(i/4) for i in channels]
        self.c1_1 = ConvBNRelu(channels=channels[0],kernel_size=1,strides=1, padding='same')
        self.c1_2 = ConvBNRelu(channels=channels[1],kernel_size=3,strides=2, padding='valid')
        self.c2_1 = ConvBNRelu(channels=channels[2],kernel_size=1,strides=1, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels[3],kernel_size=3,strides=2, padding='valid')
        self.c3_1 = ConvBNRelu(channels=channels[4],kernel_size=1,strides=1, padding='same')
        self.c3_2 = ConvBNRelu(channels=channels[5],kernel_size=3,strides=1, padding='same')
        self.c3_3 = ConvBNRelu(channels=channels[6],kernel_size=3,strides=2, padding='valid')
        self.p4_1 = MaxPool2D(pool_size=3,strides=2, padding='valid')
    def call(self,inputs):
        x1_1 = self.c1_1(inputs)
        x1_2 = self.c1_2(x1_1)
        x2_1 = self.c2_1(inputs)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(inputs)
        x3_2 = self.c3_2(x3_1)
        x3_3 = self.c3_3(x3_2)
        x4_1 = self.p4_1(inputs)
        outputs = tf.concat([x1_2,x2_2,x3_3,x4_1], axis = -1)
        return outputs

In [ ]:
#网络搭建及训练
class InceptionResnetV2(Model):
    def __init__(self):
        super(InceptionResnetV2,self).__init__()
        self.c1 = ConvBNRelu(channels=96,kernel_size=3,strides=1, padding='same')
        self.blocks = Sequential()
        for _ in range(5):
            self.blocks.add(InceptionResnetA())
        self.blocks.add(InceptionReduceA())
        for _ in range(10):
            self.blocks.add(InceptionResnetB())
        self.blocks.add(InceptionReduceB())
        for _ in range(5):
            self.blocks.add(InceptionResnetC())
        self.p1 = GlobalAveragePooling2D()
        self.d1 = Dropout(0.2)
        self.f1 = Dense(10,activation='softmax',kernel_initializer="he_normal",
                        kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))  

    def call(self,inputs):
        x = self.c1(inputs)
        x = self.blocks(x)
        x = self.p1(x)
        x = self.d1(x)
        y = self.f1(x)
        return y

In [ ]:
model = InceptionResnetV2()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=2.),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy'])

callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),  #学习率衰减表
            #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, min_lr=0.0001, patience=10, cooldown=0)
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/InceptionResnetV2_logs
#http://localhost:6006/

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 93.97%  (148epochs)